In [1]:
from pyspark.sql import SparkSession
import json

# Initialiser une session Spark
spark = SparkSession.builder.appName("ImageMetadata").getOrCreate()

# Définir le répertoire contenant les images
image_dir = 'images'

# Créer un RDD à partir de la liste des noms de fichiers d'images
image_files_rdd = spark.sparkContext.parallelize(os.listdir(image_dir))

# Filtrer les noms de fichiers d'images valides
valid_image_files_rdd = image_files_rdd.filter(lambda filename: filename.endswith((".jpg", ".png", ".jpeg")))

# Mapper pour ouvrir les images avec la bibliothèque ColorThief, extraire la couleur prédominante et enregistrer les métadonnées
def extract_image_metadata(filename):
    from colorthief import ColorThief

    image_path = os.path.join(image_dir, filename)
    color_thief = ColorThief(image_path)
    dominant_color = color_thief.get_color(quality=1)

    metadata = {
        "dominant_color": dominant_color
    }

    metadata_filename = f"metadata_{os.path.splitext(filename)[0]}.json"
    with open(metadata_filename, "w") as f:
        json.dump(metadata, f)

    return (filename, metadata_filename)

metadata_rdd = valid_image_files_rdd.map(extract_image_metadata)

# Reducer pour demander à l'utilisateur de taguer les images et enregistrer les métadonnées
def tag_images(filename_metadata):
    filename, metadata_filename = filename_metadata

    with open(metadata_filename, "r") as f:
        metadata = json.load(f)

    print(f"Veuillez taguer l'image {filename} en choisissant parmi les tags suivants :")
    print(", ".join(tags))
    user_tags = input().split()

    metadata["tags"] = user_tags

    with open(metadata_filename, "w") as f:
        json.dump(metadata, f)

# Appliquer le reducer pour demander à l'utilisateur de taguer les images et enregistrer les métadonnées
metadata_rdd.foreach(tag_images)

# Fermer la session Spark
spark.stop()


NameError: name 'ColorThief' is not defined